# 构建数据集

In [1]:
#训练集参数设置
training = 'training'#训练集文件夹名称
group_num = [90,79,171] #按windows系统排列顺序
group_name = ['control','mp','pd']
mass_rate = 0.90 #m/z检出比例

#测试集参数设置
tolerance = 0.5 #质谱峰分辨率选择，不要改动，经测试0.5比0.3好！
peak_int_learn = 0.01#根据峰强度选择质谱峰
pvalues = 0.05 #p值筛选

#数据集路径
path = 'D:\\python test\\project83\\final\\pos5\\'+training+'\\'
path_store = 'D:\\python test\\project83\\final\\pos5\\'+training+'_store\\'

# 通过pyopenms进行m/z对齐和筛选

In [2]:
import numpy as np
import pyopenms as oms
import numpy as np
from scipy import signal

#数据质控，质谱峰去噪，去平头峰
def noise_removal(prim,tolerance): 
    total = prim.values.tolist()
    ref_total = total[1:]+[[0,0]]
    new_total = [[r[0]-m[0],r[1]-m[1]] for r,m in zip(ref_total,total)]   
    tf = [total[0]]
    for new,ref,to in zip(new_total,ref_total,total):
        if new[0] >= 0.5:        
            tf = tf+[ref]
        else:
            if new[1]>=0:                      
                tf = tf[:-1]+[ref]+[ref]
            else:
                tf = tf[:-1]+[to]+[to]
    tf = [m for i,m in enumerate(tf) if m not in tf[:i]]            
    return tf

#生成openms数据格式
def openms_data_format(mass,intensity,decimal=5):
    #质谱保留
    mz = np.round(mass.values,decimal)
    mz_intensity = intensity.values
    spectrum = oms.MSSpectrum()
    spectrum.set_peaks([mz,mz_intensity])
    spectrum.sortByPosition()
    return spectrum

#质量数对齐
def mass_align(ref_spectrum,obs_spectrum,tolerance=0.5):
    alignment = []
    spa = oms.SpectrumAlignment()
    p = spa.getParameters()
    # use 0.5 Da tolerance (Note: for high-resolution data we could also use ppm 
    #by setting the is_relative_tolerance value to true)
    p.setValue("tolerance", tolerance)
    p.setValue("is_relative_tolerance", "false")
    spa.setParameters(p)
    # align both spectra
    spa.getSpectrumAlignment(alignment, ref_spectrum, obs_spectrum)
    return alignment

#取质量数平均值_1
def mass_calculation(re_spectrum,ob_spectrum,alignment,decimal=4):   
    ref = [i[0] for i in alignment]
    obs = [j[1] for j in alignment]
    ref_mass = [re_spectrum.mass[i] for i in ref]
    obs_mass = [ob_spectrum.mass[j] for j in obs]
    ave_mass = np.round((np.array(ref_mass)+np.array(obs_mass))/2,decimal)
    for i,j,q in zip(ref,obs,range(len(ave_mass))):
        re_spectrum.iloc[i, 0] = ave_mass[q]
        ob_spectrum.iloc[j, 0] = ave_mass[q]
    return re_spectrum,ob_spectrum

#按参比文件_2
def mass_calculation_ref(re_spectrum,ob_spectrum,alignment,decimal=4):   
    ref = [i[0] for i in alignment]
    obs = [j[1] for j in alignment]
    for i,j in zip(ref,obs):
        ob_spectrum.iloc[j, 0] = re_spectrum.iloc[i, 0]         
    return re_spectrum,ob_spectrum

# 初始文件的生成（默认采用第一个数据文件）

In [3]:
import os
import pandas as pd
from natsort import natsorted
from jcamp import jcamp_readfile

#读取文件列表2023年9月22日
file_list = natsorted(os.listdir(path))
column_list = [fst.split('.')[0] for fst in file_list]

#增加文件初始值2023年9月22日
name_list = file_list[1:]
col_list = column_list[1:]

#生成初始文件2023年9月22日
first_file,first_column = file_list[0],column_list[0]
prim = pd.read_excel(path+first_file)
prim = noise_removal(prim,tolerance)
prim = pd.DataFrame(prim,columns=['mass',first_column])
#prim = prim[prim[first_column] >= peak_int_learn*max_peak].reset_index(drop=True)

# 添加剩余样本

In [4]:
import numpy as np
import pandas as pd

#训练集生成20241124
for name,col in zip(name_list,col_list):
    #读取文件
    indata = pd.read_excel(path+name) 
    denoise = noise_removal(indata,tolerance)# 去除噪声
    framefile = pd.DataFrame(denoise,columns=['mass',col]) 

    #生成openms数据
    ref_spectrum = openms_data_format(prim.mass,prim.iloc[:,1])
    obs_spectrum = openms_data_format(framefile.mass,framefile.iloc[:,1])
    alignment = mass_align(ref_spectrum,obs_spectrum,tolerance)

    #数据整合
    r_spectrum,o_spectrum = mass_calculation_ref(prim,framefile,alignment)
    prim = pd.merge(prim,o_spectrum,how='outer',on='mass')#merge用好太不容易了
    prim = prim.sort_values('mass',ascending=True).reset_index(drop=True)#merge用好太不容易了
    
#最终数据的生成
outfile = prim.replace(0,np.nan).dropna(axis=0,how='all')

# 数据统计分析

In [5]:
#训练集生成和保存
prim_int = outfile

# 数据统计值计算
data_num = [0]
for p in range(len(group_num)):
    data_num.append(sum(group_num[:p+1]))

minum_num = [round(m * mass_rate) for m in group_num]
new_group_name = ['num_'+ name for name in group_name]

#生成新的数据列
for i,name in zip(range(len(data_num)-1),new_group_name):
    prim_int[name] = prim_int.iloc[:,data_num[i]+1:data_num[i+1]+1].count(axis=1)

#文件筛选
total_file = prim_int[(prim_int[new_group_name[0]] >= minum_num[0])]    
for name,mini in zip(new_group_name[1:],minum_num[1:]):
    internal_file = prim_int[(prim_int[name] >= mini)]  
    total_file = pd.merge(total_file,internal_file,how='outer') 
int_scd = total_file.copy().sort_values(by = 'mass').reset_index(drop=True)

# 数据预处理
#SimpleImputer,NaN数据插补,可选
#strategy=['mean','median','most_frequent']
#数据正则化
#'l1'特征值/征值绝对值之和、'l2'特征值/特征值绝对值平方和的开方、'max'特征值/特征值的最大值

In [6]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,Normalizer
#-----------------数据预处理：和统计分析不同------------------
makepip = make_pipeline(SimpleImputer(missing_values=np.nan,\
                                      strategy='median'),Normalizer(norm='l1'),StandardScaler())
int_sc = int_scd[column_list].T
treat = makepip.fit(int_sc)
int_sc_norm = pd.DataFrame(treat.transform(int_sc).T,columns=column_list)

# 统计分析和数据存储
#数据统计分析No.2：组间比较和统计
#9月16日修改，改用np.nanmean()求平均值
#2025.2.19修改，增加ANOVA

In [7]:
from scipy import stats
#对数据集进行拆分
internal_list = []
for i,name in zip(range(len(data_num)-1),new_group_name):
    data_internal = int_sc_norm.iloc[:,data_num[i]:data_num[i+1]].T.values
    internal_list.append(data_internal)

#判定是否需要进行anova分析
if len(group_num)>=3:
    f_statistic, p_value = stats.f_oneway(*internal_list)
    p_column = 'ANOVA'
else:
    s_statistic, p_value = stats.ttest_ind(*internal_list,equal_var=False)
    p_column = 'ttest'
    
#筛选小于p值的m/z
p_value = pd.DataFrame(p_value,columns = [p_column])
int_scf = pd.concat([int_scd,p_value],axis=1)
int_F = int_scf[(int_scf[p_column] <= 0.05)]
int_F_sort = int_F.sort_values(by = ['mass'])

#数据存储
int_F_sort.to_excel(path_store+training+'_screened_int.xlsx',index=False)
int_scf.to_excel(path_store+training+'_int.xlsx',index=False)